# FFDI and Thresholding

code is setup to process the intial bias_input_data into FFDI and thresholding.

The FFDI code uses the zarr stores of the individual simulations which requires the rechunking

Thresholding uses the FFDI files of the individual simulations


In [1]:
import sys
import os

import intake
import xarray as xr
from matplotlib import pyplot as plt
import glob
import pathlib
import traceback
from datetime import datetime
from xclim.indices import (
    keetch_byram_drought_index,
    griffiths_drought_factor,
    mcarthur_forest_fire_danger_index
)
from dask.distributed import Client

# Needed for the GWL code
from importlib import reload
# adding folder to the system path
sys.path.insert(0, '/g/data/xv83/rxm599/acs/gwls')

import gwl

In [2]:
# Start Cluster 
import dask
# Set configuration options
dask.config.set({
    'distributed.comm.timeouts.connect': '90s',  # Timeout for connecting to a worker
    'distributed.comm.timeouts.tcp': '90s',  # Timeout for TCP communications
})

client = Client()
client

2024-07-26 07:21:07,322 - distributed.preloading - INFO - Creating preload: /g/data/hh5/public/apps/dask-optimiser/schedplugin.py
2024-07-26 07:21:07,327 - distributed.utils - INFO - Reload module schedplugin from .py file
2024-07-26 07:21:07,332 - distributed.preloading - INFO - Import preload module: /g/data/hh5/public/apps/dask-optimiser/schedplugin.py


Modifying workers


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /node/gadi-mmem-clx-0003.gadi.nci.org.au/9299/proxy/8787/status,
Dashboard: /node/gadi-mmem-clx-0003.gadi.nci.org.au/9299/proxy/8787/status,Workers: 24
Total threads: 24,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44785,Workers: 24
Dashboard: /node/gadi-mmem-clx-0003.gadi.nci.org.au/9299/proxy/8787/status,Total threads: 24
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:43693,Total threads: 1
Dashboard: /node/gadi-mmem-clx-0003.gadi.nci.org.au/9299/proxy/33107/status,Memory: 0 B
Nanny: tcp://127.0.0.1:37333,


2024-07-26 07:21:35,987 - distributed.nanny - WARNING - Restarting worker
2024-07-26 07:21:35,991 - distributed.nanny - WARNING - Restarting worker
2024-07-26 07:21:35,996 - distributed.nanny - WARNING - Restarting worker
2024-07-26 07:21:36,004 - distributed.nanny - WARNING - Restarting worker
2024-07-26 07:21:36,009 - distributed.nanny - WARNING - Restarting worker
2024-07-26 07:21:36,018 - distributed.nanny - WARNING - Restarting worker
2024-07-26 07:21:36,023 - distributed.nanny - WARNING - Restarting worker
2024-07-26 07:21:36,029 - distributed.nanny - WARNING - Restarting worker
2024-07-26 07:21:36,060 - distributed.nanny - WARNING - Restarting worker
2024-07-26 07:21:36,065 - distributed.nanny - WARNING - Restarting worker
2024-07-26 07:21:36,070 - distributed.nanny - WARNING - Restarting worker
2024-07-26 07:21:36,077 - distributed.nanny - WARNING - Restarting worker
2024-07-26 07:21:36,084 - distributed.nanny - WARNING - Restarting worker
2024-07-26 07:21:36,088 - distributed.

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
catalogue_path = '/g/data/ia39/catalogues'
mRuns = sorted(glob.glob(catalogue_path + '/*ssp370*.json'))
print(len(mRuns))

13


In [6]:
def extract_model_info(filepath):
    filename = filepath.split('/')[-1]  # Get the filename from the full path
    filename_no_ext = filename.split('.')[0]  # Remove the file extension
    parts = filename_no_ext.split('_')  # Split filename by underscores
#    print(parts)
    RCM=parts[1]
    GCM=parts[2]
    
    # Extract model name
    model_name = filename_no_ext
    extension = filename.split('.')[-1]  # Get the extension
    
    match = '_'.join(parts[1:3])  # Extract match (e.g., EC-Earth3_ssp370_r1i1p1f1)
    pathway = parts[3]  # Extract model (e.g., ssp370)
    ensemble = parts[4]  # Extract ensemble (e.g., r1i1p1f1)
    
    return [model_name, extension], RCM, GCM, ensemble, pathway

In [7]:
def calculate_ffdi(pra, tasmaxa, pr_annual, hursmina, sfcWindmaxa):
    KBDI = keetch_byram_drought_index(pra, tasmaxa, pr_annual)
    DF = griffiths_drought_factor(pra, KBDI)
    FFDI = mcarthur_forest_fire_danger_index(DF, tasmaxa, hursmina, sfcWindmaxa)
    return FFDI.to_dataset(name='FFDI')

In [8]:
def process_threshold(data, time_dim, syear, eyear, var, period):
# first set time slice
    start=str(syear)+'-01-01'
    end=str(eyear)+'-12-31'
    nyear=eyear-syear+1; inyear=1./nyear
    data = data.sel(**{time_dim: slice(start, end)}).persist() # use file chunking .persist()
    print(f"Processing period {period}")
# days over key thresholds (days/yr)
    d3 = (data > 100).sum('time').to_dataset(name='days100') *inyear
    d2 = (data > 75).sum('time').to_dataset(name='days75') *inyear
    d1 = (data > 50).sum('time').to_dataset(name='days50') *inyear
    dss=xr.merge([d1,d2,d3]) 
    return dss,data

In [12]:
def process_GWL_levels(model_name, GCM, ensemble, pathway, ffdi_data):
    CMIP='CMIP6'
    GWL_levels = ['1.2', '1.5', '2.0', '3.0']
#    GWL_levels = ['1.2']
    for GWL in GWL_levels:
        print(f"Processing GWL {GWL} for model: {GCM}, ensemble: {ensemble}, pathway: {pathway}")
        try:
            start, end = gwl.get_GWL_syear_eyear(CMIP, GCM, ensemble, pathway,GWL= GWL)
        except:
            print(f"No GWL data found for {GWL}")
            continue
        ffdi_data=xr.open_zarr('/g/data/ia39/ncra/fire/'+model_name+'_FFDI.zarr')
        d4, ddata = process_threshold(ffdi_data.FFDI, 'time', start, end, 'FFDI', GWL)
        output_path = f'/g/data/ia39/ncra/fire/{model_name}_GWL{GWL}_ffdi_threshold.nc'
        output_path1 = f'/g/data/ia39/ncra/fire/ffdi/{model_name}_GWL{GWL}_ffdi.zarr'
        print(f"Saved  to {output_path}")
        d4.to_netcdf(output_path)
        ddata.to_zarr(output_path1)
    return d4

In [13]:
%%time
# iterate through all modesl
for mindex, file in enumerate(mRuns):
    print(file)
    model_name, RCM, GCM, ensemble, pathway = extract_model_info(file)
    var='FFDI'  # not used
    if mindex != 1:         #this model failed to convert
        print (mindex)
        d4=process_GWL_levels(model_name[0],GCM,ensemble,pathway,var)

    
print("Processing of all catalogues is complete.")
#client.close()

/g/data/ia39/catalogues/AGCD-05i_BOM_ACCESS-CM2_ssp370_r4i1p1f1_BARPA-R_v1-r1_day.json
0
Processing GWL 1.2 for model: ACCESS-CM2, ensemble: r4i1p1f1, pathway: ssp370
/g/data/xv83/rxm599/acs/gwls
Processing period 1.2
Saved  to /g/data/ia39/ncra/fire/AGCD-05i_BOM_ACCESS-CM2_ssp370_r4i1p1f1_BARPA-R_v1-r1_day_GWL1.2_ffdi_threshold.nc
Processing GWL 1.5 for model: ACCESS-CM2, ensemble: r4i1p1f1, pathway: ssp370
/g/data/xv83/rxm599/acs/gwls
Processing period 1.5
Saved  to /g/data/ia39/ncra/fire/AGCD-05i_BOM_ACCESS-CM2_ssp370_r4i1p1f1_BARPA-R_v1-r1_day_GWL1.5_ffdi_threshold.nc
Processing GWL 2.0 for model: ACCESS-CM2, ensemble: r4i1p1f1, pathway: ssp370
/g/data/xv83/rxm599/acs/gwls
Processing period 2.0
Saved  to /g/data/ia39/ncra/fire/AGCD-05i_BOM_ACCESS-CM2_ssp370_r4i1p1f1_BARPA-R_v1-r1_day_GWL2.0_ffdi_threshold.nc
Processing GWL 3.0 for model: ACCESS-CM2, ensemble: r4i1p1f1, pathway: ssp370
/g/data/xv83/rxm599/acs/gwls
Processing period 3.0
Saved  to /g/data/ia39/ncra/fire/AGCD-05i_BOM_A

In [ ]:
plt.subplot(2,2,1); d4.days50.plot()
plt.subplot(2,2,2); d4.days75.plot()
plt.subplot(2,2,3); d4.days100.plot()

In [ ]:
for mindex, file in enumerate(mRuns):
    print(file)

    
#    process_GWL_levels(model_name, model, ensemble, pathway, ffdi_data)

print("Processing of all catalogues is complete.")
client.close()

In [14]:
client.close()